In [2]:
!pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl (274 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [29]:
import mlflow 
import pandas as pd
import numpy as np
import mlflow.sklearn
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer , WordNetLemmatizer
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [30]:
df = pd.read_csv('data.csv')
df.sample(5)

,review,sentiment
140,THE TEMP (1993) didn't do much theatrical busi...,negative
1,"I just saw this film last night, and I have to...",positive
253,Greetings again from the darkness. Director Al...,positive
79,Wealthy widower Anthony Steffen (as Alan Cunni...,negative
336,"The numbers don't lie, 109 people have voted f...",positive


## Data Preprocessing

In [31]:
def lemminization(text):
    """help to lemmitizr text"""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

In [32]:
def remove_stop_words(text):
    """help to remove the stop words"""
    stop_words = set(stopwords.words('english'))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

    

In [33]:
def removing_numbers(text):
    """help to remove the numbers"""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

In [34]:
def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

In [35]:
def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

In [36]:
def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)


In [37]:
def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemminization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [38]:
df = normalize_text(df)
df.head()

,review,sentiment
0,every great gangster movie under current human...,positive
1,saw film last night say loved every minute tak...,positive
2,film mildly entertaining one neglect acknowled...,negative
3,quentin tarantino s partner crime roger avary ...,negative
4,sat tv hoping name would worth time but dear g...,negative


In [39]:
df['sentiment'].value_counts()

sentiment
negative    269
positive    231
Name: count, dtype: int64

In [40]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [41]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
0,every great gangster movie under current human...,1
1,saw film last night say loved every minute tak...,1
2,film mildly entertaining one neglect acknowled...,0
3,quentin tarantino s partner crime roger avary ...,0
4,sat tv hoping name would worth time but dear g...,0


In [42]:
vectorizer = CountVectorizer(max_features=500)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [44]:
import dagshub 
mlflow.set_tracking_uri('https://dagshub.com/ay747283/Chrome-Plugin.mlflow')
dagshub.init(repo_owner='ay747283', repo_name='Chrome-Plugin', mlflow=True)

## 1st experiment
mlflow.set_experiment('Logistic Regression')

2025-03-05 11:45:35,278 INFO : HTTP Request: GET https://dagshub.com/api/v1/repos/ay747283/Chrome-Plugin "HTTP/1.1 200 OK"


Initialized MLflow to track repo "ay747283/Chrome-Plugin"

2025-03-05 11:45:35,285 INFO : Initialized MLflow to track repo "ay747283/Chrome-Plugin"


Repository ay747283/Chrome-Plugin initialized!

2025-03-05 11:45:35,285 INFO : Repository ay747283/Chrome-Plugin initialized!


<Experiment: artifact_location='mlflow-artifacts:/eb797dd40c5d4e80ac383d126a3a66a3', creation_time=1741154088064, experiment_id='0', last_update_time=1741154088064, lifecycle_stage='active', name='Logistic Regression', tags={}>

In [46]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score


## configure logging
logging.basicConfig(level=logging.INFO , format="%(asctime)s %(levelname)s : %(message)s")

logging.info("Starting the experiment")

with mlflow.start_run():
    start_time = time.time()

    try:
        logging.info('logging preprocessing paramrters')
        mlflow.log_param('vectorizer' , 'Bag of Words')
        mlflow.log_param('max_features' , 500)
        mlflow.log_param('test_size' , 0.10)

        logging.info('Initiating the model')
        model = LogisticRegression(max_iter=1000)

        logging.info('Training the model')
        model.fit(X_train , y_train)
        logging.info('Model trained successfully')

        logging.info('logging model parameters')
        mlflow.log_param('model' , 'Logistic Regression')

        logging.info('model predictions')
        y_pred = model.predict(X_test)

        logging.info('logging model metrics')
        accuracy = accuracy_score(y_test , y_pred)
        precision = precision_score(y_test , y_pred)
        recall = recall_score(y_test , y_pred)
        f1 = f1_score(y_test , y_pred)

        logging.info('logging model evaluation')
        mlflow.log_metric('accuracy' , accuracy)
        mlflow.log_metric('precision' , precision)
        mlflow.log_metric('recall' , recall)
        mlflow.log_metric('f1' , f1)

        logging.info('saving and logging the model')
        mlflow.sklearn.log_model(model , 'model')

        ## log execution time
        end_time = time.time()
        logging.info(f'Execution time : {end_time - start_time}')

        ## print the results
        logging.info(f'Accuracy : {accuracy}')
        logging.info(f'Precision : {precision}')
        logging.info(f'Recall : {recall}')
        logging.info(f'F1 : {f1}')

    except Exception as e:
        logging.error(f'Error during training the model : {e}' , exc_info=True)

2025-03-05 11:46:30,631 INFO : Starting the experiment
2025-03-05 11:46:31,033 INFO : logging preprocessing paramrters
2025-03-05 11:46:32,221 INFO : Initiating the model
2025-03-05 11:46:32,221 INFO : Training the model
2025-03-05 11:46:32,242 INFO : Model trained successfully
2025-03-05 11:46:32,247 INFO : logging model parameters
2025-03-05 11:46:32,623 INFO : model predictions
2025-03-05 11:46:32,629 INFO : logging model metrics
2025-03-05 11:46:32,632 INFO : logging model evaluation
2025-03-05 11:46:34,142 INFO : saving and logging the model
2025/03/05 11:46:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-05 11:46:40,885 INFO : Execution time : 9.852190732955933
2025-03-05 11:46:40,886 INFO : Accuracy : 0.82
2025-03-05 11:46:40,886 INFO : Precision : 0.7619047619047619
2025-03-05 11:46:40,887 INFO : Recall : 0.8
2025-03-05 11:46:40,888 INFO : F

🏃 View run spiffy-ray-71 at: https://dagshub.com/ay747283/Chrome-Plugin.mlflow/#/experiments/0/runs/71be89793c404c86a1426d89b7ef6a3c
🧪 View experiment at: https://dagshub.com/ay747283/Chrome-Plugin.mlflow/#/experiments/0
